In [ ]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt

In [ ]:
split = 'chemical_cv4' # run notebook for each split of interest 
data_ef = pd.read_csv("../predictions/{}/2022_01_evaluation_all_data_python_EF.csv".format(split))
data_auc = pd.read_csv("../predictions/{}/2022_01_evaluation_all_data_python.csv".format(split))
data_hit = pd.read_csv("../predictions/toprank_{}_hitsnorm.csv".format(split)) # Those tables are generated using previous notebook
modalities_dict = {'mobc_es_op':'MO', 'cp_es_op':'CS', 'ge_es_op':'GE'}

data_ef['descriptor'].replace(modalities_dict, inplace=True)
data_auc['descriptor'].replace(modalities_dict, inplace=True)

In [ ]:
data = pd.merge(data_ef, data_auc, on=['assay_id','descriptor'])
data = pd.merge(data, data_hit, on=['assay_id', 'descriptor'])
data.drop(columns = ['AP_x', 'pr_auc_x', 'AP_y', 'pr_auc_y'], inplace = True)
data.drop_duplicates(inplace=True)
data.reset_index(inplace = True, drop = True)

In [ ]:
plt.figure(figsize=(9,6))

plt.plot(np.arange(0,0.6,0.01), 1/np.arange(0,0.6,0.01), color="red", linestyle="--")
plt.axvline(x=0.00, color="gray")
plt.axvline(x=0.015, color="green", linestyle="--")
plt.axhline(y=0.00, color="gray")
plt.ylim(-0.01,np.nanmax(data['EF1%'])+5)
plt.xlim(-0.01,0.6)

single_mod_data = data[data['descriptor'].isin(['CS','MO','GE'])]
D = single_mod_data[~(single_mod_data.auc_90_x == True) & (single_mod_data.top_rank_hit_rate > 0) & (single_mod_data.hit_rate < 0.6)]
print(len(D))
sb.scatterplot(x=D.hit_rate, y=D['EF1%'], color="gold", s=25)
D = single_mod_data[(single_mod_data.auc_90_x == True) & (single_mod_data.top_rank_hit_rate > 0)]
print(len(D))

sb.scatterplot(x=D.hit_rate, y=D['EF1%'], color="purple", s=75)
D = single_mod_data[(single_mod_data.auc_90_x == True) & (single_mod_data.top_rank_hit_rate > 0) & (single_mod_data.hit_rate < 0.01)]

print(np.mean(D['EF1%']))